In [1]:
#!/usr/bin/env python
import sys
import rospy
import numpy as np
from geometry_msgs.msg import Pose
# , Point, Quaternion
# from std_msgs.msg import String, Float32
from sdf_mp_integration.msg import WholeBodyPose
from std_msgs.msg import Int8 

class ExperimentManager:
    def __init__(self):
        self.robot_goal_pub = rospy.Publisher('full_goal', WholeBodyPose, queue_size=1)
        self.obstacle_pub = rospy.Publisher('obstacle_task', Int8, queue_size=1)

        # [x, y, z], [ox, oy, oz, ow], [joints]
        self.goals = {
            0: [[0, 0, 0], [0.0, 0.0, 0.0, 1.0], [0, 0, 1.57, 1.57, 0]], 
            1: [[3.8, 0.65, 0], [0.0, 0.0, 0.0, 1.0], [0.2, 1.2, 0.0, 0.4, 0.0]], # Grasp in far cabinet 
            2: [[1.0, 0.0, 0], [0.0, 0.0, 0.0, 1.0], [0.2, 1.2, 0.0, 0.4, 0.0]], # Grasp in midde cabinet 
            3: [[1.7, 0.0, 0], [0.0, 0.0, 0.0, 1.0], [0.0, 1.57, 0.0, 1.57, 0.0]]  # Pickup off floor
        }
        
    def go_to_start(self):
        self.robot_goal_pub.publish(self.create_robot_goal(0))

    def hide_obstacles(self):
        obs_msg = Int8()
        obs_msg.data = 0
        self.obstacle_pub.publish(obs_msg)
        
    def create_robot_goal(self, goal_id):
        goal = self.goals[goal_id]
        
        goal_msg = WholeBodyPose()
        
        goal_msg.base.header.stamp = rospy.Time.now()
        goal_msg.base.header.frame_id = "odom"

        goal_msg.base.pose.position.x = goal[0][0]
        goal_msg.base.pose.position.y = goal[0][1]
        goal_msg.base.pose.position.z = goal[0][2]
        goal_msg.base.pose.orientation.x = goal[1][0]
        goal_msg.base.pose.orientation.y = goal[1][1]
        goal_msg.base.pose.orientation.z = goal[1][2]
        goal_msg.base.pose.orientation.w = goal[1][3]
        goal_msg.arm = goal[2] 
        
        return goal_msg
        
    def start_experiment(self, goal_id, obstacle_id):
        goal_msg = self.create_robot_goal(goal_id)
    
        obs_msg = Int8()
        obs_msg.data = obstacle_id
        
        self.robot_goal_pub.publish(goal_msg)        
        self.obstacle_pub.publish(obs_msg)

    def start_experiment_no_obstacle(self, goal_id):
        goal_msg = self.create_robot_goal(goal_id)
        self.robot_goal_pub.publish(goal_msg)        

In [2]:
rospy.init_node('experiment_manager_node')
experiment_manager = ExperimentManager()

In [ ]:
experiment_manager.start_experiment_no_obstacle(1)

In [ ]:
experiment_manager.start_experiment(1, 9)

In [ ]:
experiment_manager.start_experiment(2, 10)

In [ ]:
experiment_manager.start_experiment_no_obstacle(2)

In [ ]:
experiment_manager.start_experiment(2, 11)

In [3]:
experiment_manager.start_experiment(3, 9)

In [ ]:
experiment_manager.go_to_start()

In [ ]:
obs_msg = Int8()
obs_msg.data = 9
experiment_manager.obstacle_pub.publish(obs_msg)
